##### Dependências

In [1]:
import pandas as pd
from datetime import datetime, time
import sqlite3
import unidecode

#### Importando Base de dados

In [2]:
dataset = pd.read_excel('../data/SP_capital_dadosCriminais_2025.xlsx')

##### Limpeza de dados e ordenação de colunas

In [3]:
# Utilizando somente colunas relevantes para a análise
format_dataset = dataset.iloc[:,[4,20,6,7,8,9,10,11,12]]

# Removendo as linhas duplicadas
format_dataset = format_dataset.drop_duplicates()

##### Filtragem de rubricas

**Apenas ocorrências criminais**

In [4]:
rubricas = [
    "Atropelamento",
    "Colisão",
    "Comunicação de óbito",
    "Cumprimento de mandado de busca e apreensão",
    "Encontro de Pessoa",
    "Localização/Apreensão de objeto",
    "Localização/Apreensão de veículo",
    "Localização/Apreensão e Entrega de objeto",
    "Localização/Apreensão e Entrega de veículo",
    "Morte natural",
    "Outros não criminal",
    "Perda/Extravio"
]

format_dataset['BAIRRO'] = (
    format_dataset['BAIRRO']
    .astype(str)                     
    .str.upper()                     
    .apply(unidecode.unidecode)      
    .str.strip()                     
    )

format_dataset['BAIRRO'] = format_dataset['BAIRRO'].replace('nan', None)

format_dataset = format_dataset.loc[~format_dataset.RUBRICA.isin(rubricas)]

##### Tratando relação entre período e horário de ocorrência

In [5]:
# Função para classificar periodos dos dia de acordo com seus respectivos horários.
def classificar_horario(str_time):
    try:
        hora = datetime.strptime(str_time, "%H:%M:%S").time()

        if time(6,0,0) <= hora < time(12,0,0):
            return 'Pela manhã'
        elif time(12,0,0) <= hora < time(18,0,0):
            return 'A tarde'
        elif time(18,0,0) <= hora < time(23,59,59):
            return 'A noite'
        else:
            return 'De madrugada'
    except:
        return None
    

# aplica a classificação
format_dataset.loc[format_dataset.DESC_PERIODO.isna(), 'DESC_PERIODO'] = format_dataset['HORA_OCORRENCIA_BO'].apply(classificar_horario)

# remover coluna "HORA_OCORRENCIA_BO" pois ficaria redundante
format_dataset = format_dataset.drop(columns=['HORA_OCORRENCIA_BO'])

##### Removendo dados fora do escopo

Removendo dados que estão fora do intervalo do primeiro semestre (JAN/JUN) de 2025

In [9]:
intervalo_inicio = '2025-01-01'
intervalo_fim = '2025-06-30'

format_dataset = format_dataset.loc[(format_dataset.DATA_OCORRENCIA_BO >= intervalo_inicio) & (format_dataset.DATA_OCORRENCIA_BO <= intervalo_fim)]

##### Tratamento de dataset finalizado

In [16]:
format_dataset

,NUM_BO,RUBRICA,DATA_REGISTRO,DATA_OCORRENCIA_BO,DESC_PERIODO,DESCR_SUBTIPOLOCAL,BAIRRO,LOGRADOURO
1,AM7136,Estupro de vulneravel (art.217-A),2025-01-10,2025-01-10,De madrugada,Via Pública,SE,VEDAÇÃO DA DIVULGAÇÃO DOS DADOS RELATIVOS
2,AQ9561,Estupro de vulneravel (art.217-A),2025-01-14,2025-01-14,Em hora incerta,Casa,SE,VEDAÇÃO DA DIVULGAÇÃO DOS DADOS RELATIVOS
3,BH3385,Estupro de vulneravel (art.217-A),2025-01-27,2025-01-25,A noite,Motel,CANINDE,VEDAÇÃO DA DIVULGAÇÃO DOS DADOS RELATIVOS
9,AA2520,Furto (art. 155),2025-01-01,2025-01-01,De madrugada,Via Pública,LIBERDADE,RUA JUNQUEIRA FREIRE
11,AA2974,Furto (art. 155),2025-01-01,2025-01-01,A tarde,Via Pública,SE,PARQUE DOM PEDRO II
...,...,...,...,...,...,...,...,...
234220,JH9873,Roubo (art. 157),2025-06-27,2025-06-27,A tarde,Via Pública,SAPOPEMBA,RUA SANTA MARIA DO CAMBUCA
234221,JI5078,Roubo (art. 157),2025-06-28,2025-06-28,Pela manhã,Via Pública,SAPOPEMBA,AVENIDA CASA GRANDE
234222,ID8362,Homicídio (art. 121),2025-06-05,2025-06-04,A noite,Via Pública,SAPOPEMBA,AVENIDA MANUEL PIMENTEL
234224,IW1577,"tráfico drogas (Art.33, caput)",2025-06-18,2025-06-18,A tarde,Via Pública,VILA SAO NICOLAU,RUA ARROIO REAL


Exportando para banco de dados

In [17]:
conn = sqlite3.connect('../data/crimes_sp.db')

format_dataset.to_sql('ocorrencias',conn, if_exists='replace', index=False)

226350